# Optimizing Policies for Gelderland

> This notebook formulates a 'worst case scenario' and performs a directed search for pareto set of optimized policies and reflects on the key tradeoffs of the candidate solutions.

In [1]:
from __future__ import (unicode_literals, print_function, absolute_import, division)
from ema_workbench import (Model, SequentialEvaluator, MultiprocessingEvaluator, Policy, Scenario)
from ema_workbench.em_framework.evaluators import perform_experiments
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.util import ema_logging
ema_logging.log_to_stderr(ema_logging.INFO)
import time
from prob_form import get_model_for_problem_formulation
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp
from ema_workbench.em_framework.evaluators import LHS, SOBOL, MORRIS
from ema_workbench.analysis import feature_scoring
from ema_workbench.analysis.scenario_discovery_util import RuleInductionType
from ema_workbench.em_framework.salib_samplers import get_SALib_problem
from SALib.analyze import sobol

In [2]:
dike_model, planning_steps = get_model_for_problem_formulation(7)

In [3]:
from ema_workbench import load_results

experiments_loaded, outcomes_loaded = load_results('sobol_uncertainties_sampling.tar.gz')

[MainProcess/INFO] results loaded succesfully from C:\Users\lekha\Desktop\ReferenceMDM\epa1361_open\final assignment\sobol_uncertainties_sampling.tar.gz


### Setting Reference Scenarios

From the range of uncertainties, we will create scenarios based on changing uncertainties that matter the most (Derived from sensitivity analysis, uncertainties sampling)

Dedicated Scenario: We're looking at a worst-case scenario with high chances of floods in  all the regions. To simulate this, p_fail for all the areas are taken as the 20th quantile (lower p_fail, higher chances of dike failing), flood wave shape is set to the 90th quantile to represent climate change and discount rates are set to the lowest value of 1.5 signifying higher economic damage in the future. Bmax and Brates are set as 300 and 10 respectively which is on the bad outcome side. 

In [4]:
#Worst case: Flood-inducing scenario - Global warming + High probability of ALL Dikes unable to take the hydraulic load along 
#with economic implications (through discount rates)

ref_high_dikefailure_prob = Scenario('ref_high_dikefailure_prob', **{   
    'A.0_ID flood wave shape':experiments_loaded['A.0_ID flood wave shape'].quantile(0.9),
    'A.1_Bmax':300,
    'A.1_Brate':10,
    'A.1_pfail':experiments_loaded['A.1_pfail'].quantile(0.2),
    'A.2_Bmax':300,
    'A.2_Brate':10,
    'A.2_pfail':experiments_loaded['A.2_pfail'].quantile(0.2),
    'A.3_Bmax':300,
    'A.3_Brate':10,  
    'A.3_pfail':experiments_loaded['A.3_pfail'].quantile(0.2),
    'A.4_Bmax':300,
    'A.4_Brate':10,
    'A.4_pfail':experiments_loaded['A.4_pfail'].quantile(0.2),
    'A.5_Bmax':300,
    'A.5_Brate':10,
    'A.5_pfail':experiments_loaded['A.5_pfail'].quantile(0.2),
    'discount rate 0':1.5,
    'discount rate 1':1.5,
    'discount rate 2':1.5
        })

### Search over levers with reference scenario
Directed search is most often used to search over the decision levers in order to find good candidate strategies. This is for example the first step in the Many Objective Robust Decision Making process. This is done with the workbench using the optimize method.

Epsilon is maintained as 0.1 to balance between granularity and runtime considerations. Nfe is taken as 5000. 

The result from optimize is a DataFrame with the decision variables and outcomes of interest.

In [5]:
#UNCOMMENT WHEN RUNNING  FOR THE FIRST TIME 

# with MultiprocessingEvaluator(dike_model) as evaluator:
#     results_ref1 = evaluator.optimize(nfe=5000, epsilons=[0.1]*12, searchover='levers', reference=ref_high_dikefailure_prob,
#                                )

### The optimization resulted in 1045 optimal policies which is saved as a csv file 'optimized_policies.csv'

In [ ]:
#Saving as Dataset for the next analysis - UNCOMMENT ONLY WHEN RUNNING OPTIMIZATION AND SAVING AGAIN, 
#OTHERWISE WILL OVERWRITE EXISTING FILE

# results_ref1.to_csv(r'policies_for_anmol.csv',index = False)

In [7]:
results_load = pd.read_csv('data_processed/mordm/optimized_policies.csv')

In [8]:
#Policy Levers
results_load.iloc[:,0:31].head()

0_RfR 0  0_RfR 1  0_RfR 2  1_RfR 0  1_RfR 1  1_RfR 2  2_RfR 0  2_RfR 1  \
0        0        0        0        0        0        1        1        1   
1        0        1        0        0        0        0        1        0   
2        1        0        0        1        1        0        0        1   
3        1        1        0        1        0        1        0        0   
4        0        1        1        0        1        1        0        0   

   2_RfR 2  3_RfR 0  ...  A.2_DikeIncrease 2  A.3_DikeIncrease 0  \
0        1        0  ...                  10                   0   
1        0        1  ...                   4                   9   
2        1        0  ...                   9                  10   
3        1        1  ...                   0                   5   
4        0        1  ...                   6                   3   

   A.3_DikeIncrease 1  A.3_DikeIncrease 2  A.4_DikeIncrease 0  \
0                   8                  10                  10   
1                   4                   3                  10   
2                   0                   0                   7   
3                   4                   6                   7   
4                   4                   0                   6   

   A.4_DikeIncrease 1  A.4_DikeIncrease 2  A.5_DikeIncrease 0  \
0                   6                   4                   3   
1                   5                   4                   6   
2                   3                   5                   4   
3                   1                   0                   9   
4                   4                   6                   4   

   A.5_DikeIncrease 1  A.5_DikeIncrease 2  
0                  10                   5  
1                   0                   5  
2                   2                   7  
3                   1                   2  
4                   6                   8  

[5 rows x 31 columns]

In [12]:
#Adding index for ease of plotting
results_index = results_load.reset_index()

### Plotting tradeoffs between outcomes using a Parallel coordinate plot. 

In [14]:
import plotly.express as px
# fig, ax = plt.subplots(figsize=(20,20))

fig = px.parallel_coordinates(results_index.iloc[:,32:43],color=results_index.index,
                              dimensions=results_load.iloc[:,31:42].columns
                              , width=2000, height=600)
fig.show()

### A1, A2 and A3 have high amounts of deaths which is undesirable. So, to meet Gelderland's goal of safety of citizens as an utmost priority, we will filter the optimized policies for those which has 0 deaths for the three municipalities - Doesberg, Cortenover and Zutphen. This will be used in further analysis as a limiting criteria to design better policies.

In [34]:
#Filtering policies based on absolutely zero deaths
results_zero_deaths = results_index[results_index['A.1_Expected Number of Deaths']+results_index['A.2_Expected Number of Deaths']+results_index['A.3_Expected Number of Deaths'] == 0]

In [35]:
import plotly.express as px
# fig, ax = plt.subplots(figsize=(20,20))

fig = px.parallel_coordinates(results_zero_deaths.iloc[:,32:43],color=results_zero_deaths.index,
                              dimensions=results_zero_deaths.iloc[:,32:42].columns,
                              width=1700, height=600)
fig.show()

### Once deaths are limited to zero, we see a wide difference in dike investment costs and Rfr Total Costs. 